The Geocoing with Geopy package is convient. Google API yields quite accurate data with neighborhood search. Also, Foursquare API provided with excellent venue infomation. However, the results yeild by Linear Regreassion model is not promessing, in futher experinment other model can be applied to improve the accuracy.

# Capstone Project Notebook

This project focuses on the application of data science techniques covered in the IBM Data Science Professional Certificate course wiht FourSquare API.

### By Xusen (Eason) Wang

Email: xusen@ualberta.ca \
Linked in: Xusen (Eason) Wang

### **1.Data collection and preprocessing**

In [30]:
pip install folium


In [31]:
pip install geopy

In [0]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import sklearn as sk
import folium
from geopy.geocoders import GoogleV3
import requests

#### Data summary

- Collect average housing price by neighborhood from [Toronto Open Data Base](https://open.toronto.ca)
- Collect neighborhood coordinates with Geocoder library
- Call Foursquare API for venues in the neighborhood
- Cartigorize the venues and match with respective neighborhood

In [33]:
housing_url = 'https://ckan0.cf.opendata.inter.prod-toronto.ca/download_resource/2fc77a12-fadc-447a-9ab9-1b170a508937'
housing= pd.read_excel(housing_url,"RawDataRef_2011")
housing

,Neighbourhood,Neighbourhood Id,Home Prices,Mid-Century Highrise Households,Mid-Century Highrise Population,Percent Mid-Century Highrise Households,Percent Mid-Century Highrise Population,Rent Bank Applicants,Social Housing Turnover,Social Housing Units,Social Housing Waiting List
0,West Humber-Clairville,1,317508,690,1810,6.8,5.3,38,0.312500,952,557
1,Mount Olive-Silverstone-Jamestown,2,251119,4110,13395,42.9,40.9,41,7.000000,1146,1049
2,Thistletown-Beaumond Heights,3,414216,430,1260,13.4,12.4,7,1.333333,447,270
3,Rexdale-Kipling,4,392271,600,1050,15.2,10.0,19,4.200000,432,186
4,Elms-Old Rexdale,5,233832,870,2305,27.3,24.1,14,1.000000,463,239
...,...,...,...,...,...,...,...,...,...,...,...
135,West Hill,136,308229,3010,5880,31.1,22.1,22,8.400000,2089,725
136,Woburn,137,316584,5090,14870,27.8,27.9,26,4.911765,2660,1306
137,Eglinton East,138,274020,3935,10105,48.8,44.3,33,2.588235,865,688
138,Scarborough Village,139,356096,2905,7860,49.2,47.3,22,4.400000,922,683


In [34]:
housing.columns

Index(['Neighbourhood', 'Neighbourhood Id', 'Home Prices',
       'Mid-Century Highrise Households', 'Mid-Century Highrise Population',
       'Percent Mid-Century Highrise Households',
       'Percent Mid-Century Highrise Population', 'Rent Bank Applicants',
       'Social Housing Turnover', 'Social Housing Units',
       'Social Housing Waiting List'],
      dtype='object')

In [0]:
#creating new dataframe 
price=housing[['Neighbourhood','Home Prices']]

In [36]:
len(price)
price.loc[0,'Neighbourhood']

'West Humber-Clairville'

In [0]:
#@title


In [46]:
#Use Geocode with GoogleMap API to find the coordinates of the neighbourhoods
from geopy.exc import GeocoderTimedOut
price['coordinates']=None
coor=[]
lat=[]
lng=[]
for i in range(len(price)):
    try:
        Neigh=price.loc[i,'Neighbourhood']
        query= Neigh + 'Toronto'
        results = geolocator.geocode(query)
        if results is None:
          la=0
          ln=0
        else:
          la=results.latitude
          ln=results.longitude
        lat.append(la)
        lng.append(ln)
    except GeocoderTimedOut:
        pass

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
price['Latitude']=lat
price['Longitude']=lng

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
price.tail(10)

,Neighbourhood,Home Prices,coordinates,Latitude,Longitude
130,Rouge,426850,None,43.804929,-79.165842
131,Malvern,256488,None,43.809160,-79.221690
132,Centennial Scarborough,491024,None,43.781684,-79.148327
133,Highland Creek,529278,None,43.790121,-79.173392
134,Morningside,289646,None,43.782666,-79.204997
135,West Hill,308229,None,43.769163,-79.182803
136,Woburn,316584,None,43.776470,-79.231728
137,Eglinton East,274020,None,43.724833,-79.305108
138,Scarborough Village,356096,None,43.738792,-79.217099
139,Guildwood,444309,None,43.752743,-79.192777


In [49]:
price.head(50)

,Neighbourhood,Home Prices,coordinates,Latitude,Longitude
0,West Humber-Clairville,317508,None,43.735649,-79.625844
1,Mount Olive-Silverstone-Jamestown,251119,None,43.741149,-79.589238
2,Thistletown-Beaumond Heights,414216,None,43.742903,-79.571267
3,Rexdale-Kipling,392271,None,43.719857,-79.570600
4,Elms-Old Rexdale,233832,None,43.723712,-79.548860
5,Kingsview Village-The Westway,292861,None,43.700504,-79.551914
6,Willowridge-Martingrove-Richview,511637,None,43.681927,-79.566565
7,Humber Heights-Westmount,491396,None,43.696358,-79.527033
8,Edenbridge-Humber Valley,873268,None,43.667113,-79.527974
9,Princess-Rosethorn,971668,None,43.670050,-79.547744


#### **2.Visualizating locations**

In [0]:
!wget -q -O 'toronto_crs84.geojson' https://github.com/jasonicarter/toronto-geojson/raw/master/toronto_crs84.geojson


In [0]:
to_polygon_geo_data=r'toronto_crs84.geojson'

In [53]:
to_map=folium.Map(location=[43.6532, -79.3832], zoom_start=11)
to_map.choropleth(
    geo_data=to_polygon_geo_data,
    data=price,
    columns=['Neighbourhood', 'Home Prices'],
    key_on='feature.properties.neighborhood',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Average housing price in Toronto'
)

for lat, lng, neighborhood, prices in zip(price['Latitude'], price['Longitude'], price['Neighbourhood'], price['Home Prices']):
  label = '{}, ${:3.0f}'.format(neighborhood, prices)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(to_map)


/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [54]:
to_map

### **3.Using Foursquare API to find surrounding venues**

In [0]:
#@title



In [0]:
#Foursquare parameters
radius=1000
LIMIT = 400
venues=[]
for lat, lng, neighbourhood in zip(price['Latitude'], price['Longitude'], price['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        lng,
        radius, 
        LIMIT)
    res = requests.get(url).json()['response']['groups'][0]['items']

    for venue in res:
        venues.append((
            neighbourhood,
            lat, 
            lng, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))


In [104]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueType']
print(venues_df.shape)
venues_df.head(100)


(5982, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueType
0,West Humber-Clairville,43.735649,-79.625844,Tim Hortons,43.734656,-79.617625,Coffee Shop
1,West Humber-Clairville,43.735649,-79.625844,Hwy 427 & Finch,43.733811,-79.627130,Intersection
2,West Humber-Clairville,43.735649,-79.625844,Sayona Foods,43.737692,-79.622984,Grocery Store
3,West Humber-Clairville,43.735649,-79.625844,Malton Variety,43.733146,-79.633326,Smoke Shop
4,West Humber-Clairville,43.735649,-79.625844,Indian Line Campground,43.736025,-79.635229,Campground
...,...,...,...,...,...,...,...
95,Willowridge-Martingrove-Richview,43.681927,-79.566565,Self Stor,43.685306,-79.574888,Storage Facility
96,Willowridge-Martingrove-Richview,43.681927,-79.566565,Paintball City,43.687947,-79.572022,Paintball Field
97,Willowridge-Martingrove-Richview,43.681927,-79.566565,Badlands Paintball,43.688015,-79.572515,Sporting Goods Shop
98,Humber Heights-Westmount,43.696358,-79.527033,Mayflower Chinese Food,43.692753,-79.531566,Chinese Restaurant


In [105]:
venues_type = pd.get_dummies(venues_df[['VenueType']],prefix="", prefix_sep="")
venues_type['Neighbourhood'] = venues_df['Neighbourhood']
fix_columns = list(venues_type.columns[-1:]) + list(venues_type.columns[:-1])
venues_type = venues_type[fix_columns]

print(venues_type.shape)
venues_type.head()

(5982, 348)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,...,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,West Humber-Clairville,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [106]:
venue_count_df = venues_type.groupby(['Neighbourhood']).sum().reset_index()

print(venue_count_df.shape)
venue_count_df.head()

(140, 348)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,...,Street Art,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,Agincourt South-Malvern West,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,Alderwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Annex,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,4,0,2,0,0,0,0,0,0,2,0,0,0,1,0,0,...,0,0,0,2,0,0,0,0,0,0,2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0,0,0,1
4,Banbury-Don Mills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### **4.Normalizing Data and form new data frame for further analysis**

In [222]:
from sklearn.preprocessing import StandardScaler, Normalizer, scale
scaler=StandardScaler()
N_price=scaler.fit_transform(price[['Home Prices']])
mc_data=venue_count_df
mc_data['Home Prices']=N_price
print(mc_data.shape)
mc_data.head()

(140, 349)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Shop,Bistro,Boat or Ferry,...,Supermarket,Supplement Shop,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Home Prices
0,Agincourt North,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.864931
1,Agincourt South-Malvern West,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-1.113849
2,Alderwood,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.502334
3,Annex,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,4,0,2,0,0,0,0,0,0,2,0,0,0,1,0,0,...,0,0,2,0,0,0,0,0,0,2,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0,0,0,1,-0.584615
4,Banbury-Don Mills,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-1.178665


### **5.Use regression to explore the relationship between venues and Home Prices**

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,HuberRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [0]:
X=mc_data.drop(columns=['Neighbourhood', 'Home Prices'])
y=mc_data['Home Prices']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=5)

In [0]:
lr=LinearRegression(fit_intercept=True)
reg = lr.fit(X_train, y_train)

In [0]:
y_hat=reg.predict(X_test)

In [260]:
print('R2-score:', r2_score(y_test, y_hat)) # r2 score
print('Mean Squared Error:', mean_squared_error(y_test, y_hat)) # mse

print('Max positive coefs:', lr.coef_[np.argsort(-lr.coef_)[:10]])
print('Venue types with most postive effect:', X.columns[np.argsort(-lr.coef_)[:10]].values)
print('Max negative coefs:', lr.coef_[np.argsort(lr.coef_)[:10]])
print('Venue types with most negative effect:', X.columns[np.argsort(lr.coef_)[:10]].values)
coef_abs = abs(lr.coef_)
print('Min coefs:', lr.coef_[np.argsort(coef_abs)[:10]])
print('Venue types with least effect:', X.columns[np.argsort(coef_abs)[:10]].values)

R2-score: -7.0147932894901715e+19
Mean Squared Error: 2.0058954781972595e+20
Max positive coefs: [6.12133987e+10 3.52324257e+10 2.51688682e+10 2.21000551e+10
 1.52758372e+10 1.40867701e+10 1.34244061e+10 1.19603451e+10
 1.11295810e+10 1.03574125e+10]
Venue types with most postive effect: ['Accessories Store' 'Art Museum' 'Baseball Stadium' 'Bike Shop'
 'Art Gallery' 'Campground' 'Bus Line' 'Lounge' 'Indonesian Restaurant'
 'Boat or Ferry']
Max negative coefs: [-7.10250359e+10 -5.51815055e+10 -3.13392915e+10 -2.96326653e+10
 -2.74984386e+10 -2.69215570e+10 -2.06555513e+10 -1.96728283e+10
 -1.68032281e+10 -1.62039097e+10]
Venue types with most negative effect: ['African Restaurant' 'Aquarium' 'Arcade' 'Argentinian Restaurant'
 'Amphitheater' 'Basketball Court' 'Basketball Stadium'
 'Arts & Crafts Store' 'Afghan Restaurant' 'Dessert Shop']
Min coefs: [       0.                0.                0.                0.
        0.         -6868356.24870813 40104913.14730108 43687835.8048315
 51

### **6.Conclusion**


The Geocoing with Geopy package is convient. Google API yields quite accurate data with neighborhood search. Also, Foursquare API provided with excellent venue infomation. However, the results yeild by Linear Regreassion model is not promessing, in futher experinment other model can be applied to improve the accuracy.